In [21]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler 
from langchain.prompts.example_selector.base import BaseExampleSelector

class RandomExampleSelector(BaseExampleSelector):

	def __init__(self, examples):
		self.examples = examples
	
	def add_example(self, example):
		self.examples.append(example)

	def select_examples(self, input_variable):
		from random import choice
		return [choice(examples)]


chat = ChatOpenAI(
	temperature=0.1,
	streaming=True,
	callbacks=[
		StreamingStdOutCallbackHandler()
	]
)

# 방법 1
"""
# t = PromptTemplate.from_template("What is the capital of {country}?")
# t.format(country="France")
"""

# 방법 2
"""
t = PromptTemplate(
	template="What is the capital of {country}?",
	input_variables=["country"]
)
t.format(country="France")
"""

examples = [
    {
        "question": "What do you know about France?",
				# "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
				# "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
				# "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

# Few shot prompt template
"""
example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

prompt = FewShotPromptTemplate(
	example_prompt=example_prompt,
	examples=examples,
	suffix="Human: What do you know about {country}?",
	input_variables=["country"],
)

prompt.format(country="Germany")
chain = prompt | chat
chain.invoke(
	{
		"country": "Germany"
	}
)
"""

# few shot chat template
"""
example_prompt = ChatPromptTemplate.from_messages([
	(
		"human",
		"What do you know about {country}?"
	),
	(
		"ai",
		"{answer}" 
	),
])

prompt = FewShotChatMessagePromptTemplate(
  example_prompt=example_prompt,
	examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
  (
		"system",
		"You are a geography expert"
	),
	prompt,
	(
		"human",
		"What do you know about {country}?"
	),
])

chain = final_prompt | chat
chain.invoke(
	{
		"country": "Germany"
	}
)
"""

# base example selector
example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")
example_selector = RandomExampleSelector(
	examples=examples
)
prompt = FewShotPromptTemplate(
	example_prompt=example_prompt,
	example_selector=example_selector,
	suffix="Human: What do you know about {country}?",
	input_variables=["country"],
)

prompt.format(country="Germany")


'Human: What do you know about France?\nAI: \n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Germany?'